In [9]:
from __future__ import print_function
import os
import base64
import random
import csv
import time
from lorem_text import lorem
from email.mime.text import MIMEText

from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from google.oauth2.credentials import Credentials
from google.auth.transport.requests import Request


SCOPES = ["https://www.googleapis.com/auth/gmail.send"]

def gmail_auth():
    creds = None
    if os.path.exists("token.json"):
        creds = Credentials.from_authorized_user_file("token.json", SCOPES)

    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file("email-sender-credentials.json", SCOPES)
            creds = flow.run_local_server(port=0)
        with open("token.json", "w") as token:
            token.write(creds.to_json())

    return build("gmail", "v1", credentials=creds)


# leer csv con datos sintéticos 
def cargar_csv(csv_path="muestra.csv"):
    registros = []
    with open(csv_path, newline="", encoding="utf-8") as f:
        reader = csv.DictReader(f)
        for row in reader:
            registros.append(row)
    return registros


# selección aleatoria y campos aleatorios
def elegir_datos_aleatorios(registros):
    # elegir entre 1 y 6 registros
    n_registros = random.randint(1, min(3, len(registros)))
    seleccionados = random.sample(registros, n_registros)

    datos_finales = []

    for reg in seleccionados:
        # elegir entre 1 y 6 columnas aleatorias del registro
        columnas = ["documento", "telefono", "email",
                    "cuenta_bancaria", "nombre", "ciudad"]

        n_campos = random.randint(1, 6)
        campos_aleatorios = random.sample(columnas, n_campos)

        # guardar solo los datos individuales elegidos
        for campo in campos_aleatorios:
            valor = reg[campo]
            datos_finales.append(valor)

    return datos_finales


# insertar datos en texto
def insertar_datos_en_texto(datos):
    texto = lorem.paragraphs(3)
    palabras = texto.split()

    for valor in datos:
        pos = random.randint(0, len(palabras)-1)
        palabras.insert(pos, valor)

    return " ".join(palabras)


# enviar email
def enviar_email(service, destinatario, asunto, mensaje):
    mime_message = MIMEText(mensaje)
    mime_message["to"] = destinatario
    mime_message["subject"] = asunto

    raw = base64.urlsafe_b64encode(mime_message.as_bytes()).decode()
    message = {"raw": raw}

    return service.users().messages().send(userId="me", body=message).execute()


# main
if __name__ == "__main__":
    print("Autenticando con Gmail...")
    service = gmail_auth()

    registros = cargar_csv("muestra.csv")

    total_correos = 50
    retraso_minutos = 2  
    retraso_segundos = retraso_minutos * 60

    for i in range(1, total_correos + 1):
        datos_sueltos = elegir_datos_aleatorios(registros)
        cuerpo = insertar_datos_en_texto(datos_sueltos)
        enviado = enviar_email(
            service,
            destinatario="main.dupont.tfg@gmail.com",
            asunto=f"correo con datos sensibles #{i}",
            mensaje=cuerpo
        )
        print(f"[{i}/{total_correos}] Correo enviado. ID: {enviado['id']}")
        if i < total_correos:
            print(f"Esperando {retraso_minutos} minutos antes del siguiente correo...")
            time.sleep(retraso_segundos)


Autenticando con Gmail...
[1/50] Correo enviado. ID: 19a91a553789ad48
Esperando 2 minutos antes del siguiente correo...
[2/50] Correo enviado. ID: 19a91a72b6152b79
Esperando 2 minutos antes del siguiente correo...
[3/50] Correo enviado. ID: 19a91a9003acce3e
Esperando 2 minutos antes del siguiente correo...
[4/50] Correo enviado. ID: 19a91aad6b7498af
Esperando 2 minutos antes del siguiente correo...
[5/50] Correo enviado. ID: 19a91acae6153496
Esperando 2 minutos antes del siguiente correo...
[6/50] Correo enviado. ID: 19a91ae84ff9a047
Esperando 2 minutos antes del siguiente correo...
[7/50] Correo enviado. ID: 19a91b05f7d39f4a
Esperando 2 minutos antes del siguiente correo...
[8/50] Correo enviado. ID: 19a91b23bcf08e3b
Esperando 2 minutos antes del siguiente correo...
[9/50] Correo enviado. ID: 19a91b41250c911e
Esperando 2 minutos antes del siguiente correo...
[10/50] Correo enviado. ID: 19a91b5ecec385ce
Esperando 2 minutos antes del siguiente correo...
[11/50] Correo enviado. ID: 19a91